In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scanpy as sc
import squidpy as sq
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
from anndata import AnnData

In [2]:
coordinates = pd.read_csv("./scDesign3/location.csv", index_col=0)

In [3]:
for num_de in [50, 100, 150, 200]:
    df_counts = pd.read_csv(f'./scDesign3/counts_svg_{num_de}.csv', index_col=0).transpose()
    df_true_pos = pd.read_csv(f'./scDesign3/labels_svg_{num_de}.csv', index_col=0)
    df_true_neg = pd.read_csv(f'./scDesign3/labels_non_svg_{num_de}.csv', index_col=0)
    
    df_true_pos['spatially_variable'] = True
    df_true_neg['spatially_variable'] = False
    
    df_true = pd.concat([df_true_pos, df_true_neg])
    df_true.set_index('x', inplace=True)
    
    df_true = df_true.loc[df_counts.columns, ]
    
    obs = pd.DataFrame(data={"cells": df_counts.index}, index=df_counts.index)    
    counts = sp.sparse.csr_matrix(df_counts.values)
    
    adata = AnnData(counts, obs=obs, obsm={"spatial": coordinates.values}, var=df_true, dtype=np.float32)

    adata.layers['counts'] = adata.X.copy()
    spatial_key = "spatial"
    library_id = "tissue42"
    adata.uns[spatial_key] = {library_id: {}}
    
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    
    adata.write_h5ad(f'./scDesign3/svg_{num_de}.h5ad')